In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
from qiskit_machine_learning.neural_networks import EstimatorQNN,SamplerQNN
from IPython.display import clear_output
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
from qiskit.circuit import ParameterVector
from qiskit_algorithms.utils import algorithm_globals
import time as t
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2,RealAmplitudes
import numpy as np
from qiskit.primitives import Estimator
import pickle 

from qiskit.algorithms.gradients import ParamShiftEstimatorGradient

In [ ]:
VQE_training = True
n_qubits=4
j_coupling=1
g_coupling=1.5
n_rep_VQEansatz=1

def callback_graph(x,name='loss'):
    #clear_output(wait=True)
    plt.title("Training curve")
    plt.xlabel("Iteration")
    plt.ylabel(name)
    plt.plot(range(len(x)), x,label='ground state energy')
    #plt.show()


In [ ]:
h_coupling=j_coupling*1
qr_VQE = QuantumRegister(n_qubits)
qc_VQE = QuantumCircuit(qr_VQE)
ansatz_VQE =  RealAmplitudes(num_qubits=n_qubits,reps=n_rep_VQEansatz,parameter_prefix='w')
qc_VQE.h(qr_VQE)
qc_VQE.compose(ansatz_VQE, inplace=True)  
#qc_VQE.decompose().draw('mpl')


In [ ]:
observable = SparsePauliOp.from_list([("X"+"I"*(n_qubits-1) , h_coupling )])
for i in np.arange(1,n_qubits,1):
    observable=observable + SparsePauliOp.from_list([("I"*(i)+"X"+"I"*(n_qubits-1-i) , -h_coupling )])
for i in range(n_qubits-1):
    observable=observable + (SparsePauliOp.from_list([("I"*(i)+"ZZ"+"I"*(n_qubits-2-i) ,- j_coupling)]))
    
eigenValues=np.linalg.eig(observable.to_matrix())[0]
eigenValues.sort()
groundEnergy=eigenValues[0]
print("ground state energy =", groundEnergy)


qnn_VQE=EstimatorQNN(
        estimator=Estimator(options={"shots":1e10}),
        circuit=qc_VQE,
        weight_params=ansatz_VQE.parameters,
        observables=observable
        )    

In [ ]:
init_weights=np.random.rand(qnn_VQE.num_weights)*2*np.pi
weights=init_weights

estimator = Estimator()
job=estimator.run(circuits=[qc_VQE], observables=[observable]
                          , parameter_values=[weights])

gradEstimator= ParamShiftEstimatorGradient(estimator )
gradJob=gradEstimator.run(circuits=[qc_VQE], observables=[observable]
                          , parameter_values=[weights])

In [ ]:
job.result()


In [ ]:
gradJob.result().gradients

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
service = QiskitRuntimeService()

In [ ]:
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Estimator
from qiskit.providers.fake_provider import FakeManila

backend_Osaka=service.get_backend('ibm_osaka')
noise_model = NoiseModel.from_backend(backend_Osaka)

simulator = service.get_backend('ibmq_qasm_simulator')
simulator.set_options(noise_model=noise_model)

simulator_noNoise = service.get_backend('ibmq_qasm_simulator')

In [ ]:
# Set options to include the noise model with error mitigation
options0= Options(resilience_level=0)
options1= Options(resilience_level=1)
#options.optimization_level = 0 # no optimization
#options.resilience_level = 0 # M3 for Sampler and T-REx for Estimator


In [ ]:
nEvents=1
nShots=2**13
print(nShots)

In [ ]:
learning_rate=1e-1
init_weights=np.random.rand(qnn_VQE.num_weights)*2*np.pi
weights=init_weights

loss_list=[]
jobs_list=[]
gradJobs_list=[]

with Session(service=service, backend=simulator ) as session:
    estimator = Estimator(session=session,options=options0)
    for i in range(50):
        job=estimator.run(circuits=[qc_VQE], observables=[observable]
                          , parameter_values=[weights]
                          ,shots=nShots)


        gradEstimator= ParamShiftEstimatorGradient(estimator )
        gradJob=gradEstimator.run(circuits=[qc_VQE], observables=[observable]
                              , parameter_values=[weights]
                              ,shots=nShots)

        jobs_list.append(job)
        gradJobs_list.append(gradJob)
        loss=job.result().values[0]
        loss_list.append(loss)
        weights=weights-learning_rate*gradJob.result().gradients[0]

        print("loss=",loss)

In [ ]:
learning_rate=1e-1
weights=init_weights

loss_list_noNoise=[]
jobs_list_noNoise=[]
gradJobs_list_noNoise=[]

with Session(service=service, backend=simulator_noNoise ) as session:
    estimator = Estimator(session=session,options=options0)
    for i in range(50):
        job=estimator.run(circuits=[qc_VQE], observables=[observable]
                          , parameter_values=[weights]
                          ,shots=nShots)


        gradEstimator= ParamShiftEstimatorGradient(estimator )
        gradJob=gradEstimator.run(circuits=[qc_VQE], observables=[observable]
                              , parameter_values=[weights]
                              ,shots=nShots)

        jobs_list_noNoise.append(job)
        gradJobs_list_noNoise.append(gradJob)
        loss=job.result().values[0]
        loss_list_noNoise.append(loss)
        weights=weights-learning_rate*gradJob.result().gradients[0]

        print("loss=",loss)

In [ ]:
plt.figure(1)
LabelSize=30
plt.figure(figsize=(16,9))
plt.plot(range(len(loss_list)),loss_list,label='simulator (Osaka noise model)')
plt.plot(range(len(loss_list_noNoise)),loss_list_noNoise,label='simulator (No noise)')
plt.plot(range(len(loss_list)),groundEnergy*np.ones(len(loss_list)),label='truth')
plt.xlabel('Iterations',fontsize=LabelSize)
plt.ylabel('Ground state energy',fontsize=LabelSize)
plt.tick_params(axis='both', which='major', labelsize=LabelSize)
plt.legend(fontsize=LabelSize)